In [2]:
import cv2
import numpy as np

# Load YOLO
net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")  # Provide the paths to the YOLO weights and configuration files
layer_names = net.getLayerNames()
output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]

# Initialize variables
people_in = 0
people_out = 0
first_frame = None

# Initialize video capture
cap = cv2.VideoCapture(0)  

while True:
    # Read the current frame from the video stream
    ret, frame = cap.read()

    if not ret:
        break

    # Resize the frame to improve performance
    frame = cv2.resize(frame, None, fx=0.4, fy=0.4)

    # Detect objects using YOLO
    height, width, channels = frame.shape
    blob = cv2.dnn.blobFromImage(frame, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    outs = net.forward(output_layers)

    # Initialize lists for storing detected person bounding boxes and centroids
    boxes = []
    centroids = []

    # Process each output layer
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]

            # Check if the detected object is a person and has sufficient confidence
            if class_id == 0 and confidence > 0.5:
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)

                # Calculate the top-left coordinates of the bounding box
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)

                # Store the bounding box and centroid coordinates
                boxes.append([x, y, w, h])
                centroids.append((center_x, center_y))

                # Draw bounding boxes and centroids on the frame
                cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
                cv2.circle(frame, (center_x, center_y), 2, (0, 0, 255), -1)

    # Count people entering and exiting the store
    if first_frame is None:
        first_frame = centroids
    else:
        for centroid in centroids:
            # Check if the centroid is within a certain distance of the first frame centroids
            distances = [np.linalg.norm(np.array(centroid) - np.array(c)) for c in first_frame]

            if min(distances) > 50:  # Change the threshold distance as per your requirement
                if centroid[0] < width / 2:
                    people_in += 1
                else:
                    people_out += 1

        # Update the first frame centroids
        first_frame = centroids

    # Display the current count on the frame
    cv2.putText(frame, f"People In: {people_in}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)
    cv2.putText(frame, f"People Out: {people_out}", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)

    # Display the resulting frame
    cv2.imshow('People Counter', frame)

    # Exit the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video stream and close all windows
cap.release()
cv2.destroyAllWindows()


ValueError: min() arg is an empty sequence